### Analyse et comprehension des fichiers

In [ ]:
import matplotlib.pyplot as plt
from read_data import *
from utils import *
from fasttext_utils import *
from sklearn.model_selection import train_test_split
import fasttext

In [ ]:
store_full_dataset_with_features(from_scratch=False, vectorize=False)

In [ ]:
train, test = get_processed_data()

In [ ]:
d = get_clustering_coef(test["author"])

In [ ]:
get_all_number_of_coauthors_with_hindex(train["author"])

**distribution des données**

In [ ]:
train,n_train = get_train_data()
test,n_test = get_test_data()

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(x=train['hindex'],bins=187);

**selection of sub-dataset with same distribution than the original**

**Tout le dataset (Préalablement enregistré en csv)**

In [ ]:
# store_whole_dataset(train, "../tmp/train") # si tu n'as pas encore tout le dataset en local
# store_whole_dataset(test, "../tmp/test")

In [ ]:
data = pd.read_csv("../tmp/train_full.csv")
data.head()

**Juste 10000 données**

In [ ]:
data_path = "../tmp/data_10000.csv"
# data = preprocessing_for_fastText(10000,train)
# data.to_csv(data_path,index=None)
data = pd.read_csv(data_path)
data = small_class(data, 6)
data.head()

In [ ]:
data.describe()

In [ ]:
train1,test1 = train_test_split(data,random_state=1)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1).hist(train1['hindex_lab'])
plt.subplot(1,2,2).hist(test1['hindex_lab']);

In [ ]:
train_path = "../tmp/train1.txt"
test_path = "../tmp/test1.txt"

df_to_txt(train1,train_path)
df_to_txt(test1,test_path)

### Utilisation de Fast text

Les paramètre optimaux ont été optimisés pour 10000 données dans un autre jupyter avec un temps long. Le model 1 nest plus nécessaire!

In [ ]:
model0 = fasttext.train_supervised(train_path,lr = 0.626905, dim = 12, epoch = 11, wordNgrams =3)

In [ ]:
model2 =  fasttext.train_supervised(train_path,lr = 0.15815, dim = 2, epoch = 33, wordNgrams =3)

In [ ]:
# train_duration = 60 # in seconds
#model1 = fasttext.train_supervised(train_path, autotuneValidationFile=test_path)

In [ ]:
model0.test(test_path)

In [ ]:
 model2.test(test_path)

In [ ]:
comp = pd.DataFrame({'paramètre':['hidden layer','learning rate','epoch','loss','ngram'],'model0':[model0.dim,model0.lr,model0.epoch,model0.loss,model0.wordNgrams],'model1':[model1.dim,model1.lr,model1.epoch,model1.loss,model0.wordNgrams]})
comp

**exploitation de model**

On est sur que le modèle applique softmax sur la représentation du mot. Nous ou choisit donc d'appliquer un autre régresseur

**Data for multiple regression model**

In [ ]:
X_train, y_train, X_test, y_test = get_numpy_data(n=174241)

In [ ]:
X_train_norm, X_test_norm = normalize(X_train, X_test)

**Random forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

**XG_BOOST**

In [ ]:
model_xg = XGBRegressor(learning_rate=0.01, max_depth=5, objective="reg:linear", subsample=0.7, n_estimators=500)
model_xg.fit(X_train,y_train)
mod_preds = model_xg.predict(X_test)
mean_squared_error(y_test,mod_preds)

**Neural Network**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
model_nn = Sequential()
model_nn.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model_nn.add(Dense(units=10))
model_nn.add(Dense(units=1))
model_nn.fit(X_train,y_train, epochs=10, batch_size=5)
mod_nn_preds = model_nn.predict(X_test)
print(len(mod_nn_preds))
mean_squared_error(y_test,mod_nn_preds)

**Lasso linear**

In [ ]:
from sklearn import linear_model
model_lasso = linear_model.Lasso(alpha=0.75, fit_intercept=False, tol=0.000000001,max_iter=10000000, positive=True)
model_lasso.fit(X_train,y_train)
mod_lasso_preds = model_lasso.predict(X_test)
mean_squared_error(y_test,mod_lasso_preds)

**Decision tree**

In [ ]:
from sklearn import tree
model_tree = tree.DecisionTreeRegressor()
model_tree.fit(X_train,y_train)
mod_tree_preds = model_tree.predict(X_test)
mean_squared_error(y_test,mod_tree_preds)

### CATBOOST

In [ ]:
from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test_norm)
mean_squared_error(y_test,mod_preds)

**Light GBM**

In [ ]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label = y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'mse'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)
y_pred=clf.predict(X_test)
mean_squared_error(y_test,y_pred)

**test linear regression**

In [ ]:
from sklearn import linear_model

reg = linear_model.TweedieRegressor().fit(X_train,y_train)

In [ ]:
reg_pred = reg.predict(X_test)

In [ ]:
mean_squared_error(y_test,reg_pred)

**test svm**

In [ ]:
from sklearn.svm import SVC

model_svm =SVC(kernel='sigmoid', C=2)

In [ ]:
model_svm.fit(X_train,y_train)

In [ ]:
svm_pred = model_svm.predict(X_test)

In [ ]:
mean_squared_error(y_test,svm_pred)

**Test CNN**

# Soumission

In [ ]:
from submit import submit
# from submit2 import submit2
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
# from catboost import CatBoostRegressor

model_cat = CatBoostRegressor(verbose=False)
forest_model = RandomForestRegressor()
model_xg = XGBRegressor(learning_rate=0.01, max_depth=5, objective="reg:linear", subsample=0.7, n_estimators=500)

submit(model_cat)